In [ ]:
import os

VESPA_TOKEN = os.getenv("VESPA_WORKSHOP_TOKEN", input("Enter your Vespa Cloud token: "))
VESPA_URL = os.getenv(
    "VESPA_URL",
    input("Enter your Vespa Cloud URL (e.g. https://instance.vespa-app.cloud): "),
)

In [ ]:
from vespa.application import Vespa

app = Vespa(
    url=VESPA_URL,
    vespa_cloud_secret_token=VESPA_TOKEN,
)

In [24]:
app.get_application_status()

<Response [200]>

In [25]:
from pprint import pprint

## Product query examples

In [26]:
# 1) Text search only
resp = app.query(
    yql="select * from product.product where userQuery()",
    query="chocolate",
    **{"ranking.profile": "product_search"},
)
pprint(resp.json)

{'root': {'children': [{'fields': {'aisle': 'cookies cakes',
                                   'department': 'snacks',
                                   'documentid': 'id:products:product::1',
                                   'product_id': 1,
                                   'product_name': 'Chocolate Sandwich Cookies',
                                   'sddocname': 'product',
                                   'summaryfeatures': {'closeness(field,embedding)': 0.0,
                                                       'nativeRank(product_name)': 0.38186238359951247,
                                                       'vespa.summaryFeatures.cached': 0.0}},
                        'id': 'id:products:product::1',
                        'relevance': 0.38186238359951247,
                        'source': 'product'}],
          'coverage': {'coverage': 100,
                       'documents': 10,
                       'full': True,
                       'nodes': 1,
            

In [27]:
# 2) Text search with filter
resp = app.query(
    yql="select * from product.product where userQuery() AND product_id=1",
    query="chocolate",
    **{"ranking.profile": "product_search"},
)
pprint(resp.json)

{'root': {'children': [{'fields': {'aisle': 'cookies cakes',
                                   'department': 'snacks',
                                   'documentid': 'id:products:product::1',
                                   'product_id': 1,
                                   'product_name': 'Chocolate Sandwich Cookies',
                                   'sddocname': 'product',
                                   'summaryfeatures': {'closeness(field,embedding)': 0.0,
                                                       'nativeRank(product_name)': 0.38186238359951247,
                                                       'vespa.summaryFeatures.cached': 0.0}},
                        'id': 'id:products:product::1',
                        'relevance': 0.38186238359951247,
                        'source': 'product'}],
          'coverage': {'coverage': 100,
                       'documents': 10,
                       'full': True,
                       'nodes': 1,
            

In [28]:
# 3) Hybrid search (text + vector)
resp = app.query(
    yql=(
        "select * from product.product where "
        "userQuery() OR ({targetHits: 10}nearestNeighbor(embedding, q_emb))"
    ),
    query="chocolate",
    **{
        "input.query(q_emb)": "embed(@query)",  # use your app’s embedder
        "ranking.profile": "product_search",
    },
)
pprint(resp.json)

{'root': {'children': [{'fields': {'aisle': 'cookies cakes',
                                   'department': 'snacks',
                                   'documentid': 'id:products:product::1',
                                   'product_id': 1,
                                   'product_name': 'Chocolate Sandwich Cookies',
                                   'sddocname': 'product',
                                   'summaryfeatures': {'closeness(field,embedding)': 0.6519131952419696,
                                                       'nativeRank(product_name)': 0.38186238359951247,
                                                       'vespa.summaryFeatures.cached': 0.0}},
                        'id': 'id:products:product::1',
                        'relevance': 1.033775578841482,
                        'source': 'product'},
                       {'fields': {'aisle': 'yogurt',
                                   'department': 'dairy eggs',
                                   

## User query examples

In [29]:
# 4) Fetch user by ID
resp = app.query(yql="select * from user where user_id=1")
pprint(resp.json)

{'root': {'children': [{'fields': {'documentid': 'id:users:user::1',
                                   'sddocname': 'user',
                                   'user_id': 1,
                                   'user_profile': {'10258': 1,
                                                    '13032': 1,
                                                    '196': 1,
                                                    '25133': 1,
                                                    '26088': 1,
                                                    '26405': 1,
                                                    '27845': 1,
                                                    '38928': 1,
                                                    '39657': 1,
                                                    '46149': 1,
                                                    '49235': 1},
                                   'user_purchases': [{'order_dow': 4,
                                                    

In [30]:
# 5) Find users who purchased a specific product using WAND
resp = app.query(
    yql="select * from user where ({targetHits: 25}wand(user_profile, [[196,1]]))",
    ranking="unranked",  # disable ranking; retrieve targetHits
)
pprint(resp.json)

{'root': {'children': [{'fields': {'documentid': 'id:users:user::1',
                                   'sddocname': 'user',
                                   'user_id': 1,
                                   'user_profile': {'10258': 1,
                                                    '13032': 1,
                                                    '196': 1,
                                                    '25133': 1,
                                                    '26088': 1,
                                                    '26405': 1,
                                                    '27845': 1,
                                                    '38928': 1,
                                                    '39657': 1,
                                                    '46149': 1,
                                                    '49235': 1},
                                   'user_purchases': [{'order_dow': 4,
                                                    

In [31]:
# 6) Users who purchased multiple products (WAND)
resp = app.query(
    yql="select * from user where ({targetHits: 25}wand(user_profile, [[196,1],[25133,1]]))",
    ranking="unranked",
)
pprint(resp.json)

{'root': {'children': [{'fields': {'documentid': 'id:users:user::1',
                                   'sddocname': 'user',
                                   'user_id': 1,
                                   'user_profile': {'10258': 1,
                                                    '13032': 1,
                                                    '196': 1,
                                                    '25133': 1,
                                                    '26088': 1,
                                                    '26405': 1,
                                                    '27845': 1,
                                                    '38928': 1,
                                                    '39657': 1,
                                                    '46149': 1,
                                                    '49235': 1},
                                   'user_purchases': [{'order_dow': 4,
                                                    

In [32]:
# 7) Rank users by sparse dot product of profile vs query vector
resp = app.query(
    yql="select * from user where ({targetHits: 25}wand(user_profile, [[196,1],[25133,1]]))",
    ranking="sparse_dot",  # assumes a 'sparse_dot' profile is defined in your app
)
pprint(resp.json)

{'root': {'children': [{'fields': {'documentid': 'id:users:user::1',
                                   'sddocname': 'user',
                                   'user_id': 1,
                                   'user_profile': {'10258': 1,
                                                    '13032': 1,
                                                    '196': 1,
                                                    '25133': 1,
                                                    '26088': 1,
                                                    '26405': 1,
                                                    '27845': 1,
                                                    '38928': 1,
                                                    '39657': 1,
                                                    '46149': 1,
                                                    '49235': 1},
                                   'user_purchases': [{'order_dow': 4,
                                                    

## Order query examples

In [33]:
# 8) Fetch order by ID
resp = app.query(yql="select * from order where order_id=1187899")
pprint(resp.json)

{'root': {'children': [{'fields': {'days_since_prior_order': 14.0,
                                   'documentid': 'id:orders:order::1187899',
                                   'order_dow': 4,
                                   'order_hour_of_day': 8,
                                   'order_id': 1187899,
                                   'product_ids': [196,
                                                   25133,
                                                   38928,
                                                   26405,
                                                   39657,
                                                   10258,
                                                   13032,
                                                   26088,
                                                   27845,
                                                   49235,
                                                   46149],
                                   'sddocname': 'orde

In [34]:
# 9) Find orders containing a specific product
resp = app.query(yql='select * from order where product_ids contains "196"')
pprint(resp.json)

{'root': {'children': [{'fields': {'days_since_prior_order': 14.0,
                                   'documentid': 'id:orders:order::1187899',
                                   'order_dow': 4,
                                   'order_hour_of_day': 8,
                                   'order_id': 1187899,
                                   'product_ids': [196,
                                                   25133,
                                                   38928,
                                                   26405,
                                                   39657,
                                                   10258,
                                                   13032,
                                                   26088,
                                                   27845,
                                                   49235,
                                                   46149],
                                   'sddocname': 'orde

In [35]:
# 10) Find orders containing multiple specific products
resp = app.query(
    yql='select * from order where product_ids contains "196" AND product_ids contains "25133"'
)
pprint(resp.json)

{'root': {'children': [{'fields': {'days_since_prior_order': 14.0,
                                   'documentid': 'id:orders:order::1187899',
                                   'order_dow': 4,
                                   'order_hour_of_day': 8,
                                   'order_id': 1187899,
                                   'product_ids': [196,
                                                   25133,
                                                   38928,
                                                   26405,
                                                   39657,
                                                   10258,
                                                   13032,
                                                   26088,
                                                   27845,
                                                   49235,
                                                   46149],
                                   'sddocname': 'orde